In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

DATA_INTERIM = Path("../data/interim")
DATA_PROCESSED = Path("../data/processed")
DATA_PROCESSED.mkdir(parents=True, exist_ok=True)


In [2]:
product = pd.read_csv(DATA_INTERIM / "product_clean.csv")
sales = pd.read_csv(DATA_INTERIM / "sales_clean.csv")
discount = pd.read_csv(DATA_INTERIM / "discount_clean.csv")
marketing = pd.read_csv(DATA_INTERIM / "marketing_clean.csv")

print(product.shape, sales.shape, discount.shape, marketing.shape)


(50, 3) (25976, 5) (4150, 4) (494, 3)


In [3]:
df = sales.merge(
    product,
    on="product_id",
    how="left",
    validate="many_to_one"  # many sales rows map to one product
)

df.shape, df.head()


((25976, 7),
    week store_id product_id  units_sold  selling_price   category  cost_price
 0     1      S01       P001        17.0          51.95      Dairy       31.77
 1     1      S01       P002         7.0          38.57  Household       26.05
 2     1      S01       P003        25.0          34.52  Beverages       23.56
 3     1      S01       P004        29.0          35.43     Snacks       22.08
 4     1      S01       P005        19.0          19.55      Fresh       14.62)

In [4]:
df = df.merge(
    discount,
    on=["week", "store_id", "product_id"],
    how="left",
    validate="one_to_one"
)

df.shape


(25976, 8)

In [5]:
df["discount_percent"] = df["discount_percent"].fillna(0)


In [6]:
df = df.merge(
    marketing,
    on=["week", "store_id"],
    how="left",
    validate="many_to_one"
)

df.shape


(25976, 9)

In [7]:
df["marketing_spend"] = (
    df.groupby("store_id")["marketing_spend"]
      .transform(lambda x: x.fillna(x.median()))
)


# Creating Important KPI's

In [9]:
df["final_price"] = df["selling_price"] * (1 - df["discount_percent"] / 100)
df["revenue"] = df["units_sold"] * df["final_price"]
df["profit"] = df["units_sold"] * (df["final_price"] - df["cost_price"])



In [10]:
df[["final_price", "revenue", "profit"]].describe()


,final_price,revenue,profit
count,25976.000000,25976.000000,25976.000000
mean,34.829205,731.382234,181.060398
std,10.215507,481.462309,132.866874
min,9.443500,0.000000,-382.725000
25%,27.403750,384.000000,92.535000
50%,34.730500,610.180000,148.390000
75%,41.880000,952.570000,233.661250
max,60.090000,4022.250000,1526.250000


In [13]:
df.to_csv(DATA_PROCESSED / "model_data.csv", index=False)
print("Saved model_data.csv")


Saved model_data.csv
